In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['TG']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('triglyceride', 5915),
 ('thapsigargin', 939),
 ('triacylglycerol', 510),
 ('transgenic', 505),
 ('trigeminal ganglion', 235),
 ('trigeminal ganglia', 162),
 ('thyroglobulin', 127),
 ('thrombin generation', 108),
 ('transglutaminase', 91),
 ('thioguanine', 50),
 ('thioglycollate', 43),
 ('triacylglyceride', 34),
 ('trophoblast giant', 26),
 ('triglycide', 22),
 ('transplant glomerulopathy', 20),
 ('target gene', 18),
 ('tripterygium glycosides', 18),
 ('trigeminal', 15),
 ('total glutathione', 14),
 ('total gastrectomy', 14),
 ('treatment group', 13),
 ('transglycosylation', 12),
 ('training group', 11),
 ('total ginsenosides', 10),
 ('troglitazone', 10),
 ('tg', 9),
 ('tg101209', 9),
 ('thymine glycol', 8),
 ('transient grating', 7),
 ('typhus group', 7),
 ('triacyglycerol', 7),
 ('toxoplasma gondii', 5),
 ('timothy grass', 5),
 ('triglyeride', 5),
 ('thioglycerol', 5),
 ('tara gum', 4),
 ('thyroid gland', 4),
 ('triglycéride', 4),
 ('trigonelline', 4),
 ('therapy group', 4),
 ('thy

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:17855': 'Triglyceride',
 'CHEBI:CHEBI:9516': 'Thapsigargin',
 'EFO:0004020': 'transgenic',
 'MESH:D012668': 'Trigeminal Ganglion',
 'HGNC:11764': 'TG',
 'EFO:0000507': 'generation',
 'CHEBI:CHEBI:9555': 'tioguanine',
 'CHEBI:CHEBI:30066': 'thioglycolate(1-)',
 'HP:HP:0100820': 'Glomerulopathy',
 'CHEBI:CHEBI:24400': 'glycoside',
 'CHEBI:CHEBI:16856': 'glutathione',
 'MESH:D005743': 'Gastrectomy',
 'CHEBI:CHEBI:24433': 'group',
 'CHEBI:CHEBI:74978': 'ginsenoside',
 'CHEBI:CHEBI:9753': 'troglitazone',
 'CHEBI:CHEBI:90304': 'TG101209',
 'CHEBI:CHEBI:29128': 'thymine glycol',
 'MESH:D031784': 'Phleum',
 'MESH:C009465': 'thioglycerol',
 'MESH:C038612': 'tara gum',
 'MESH:D013961': 'Thyroid Gland',
 'MESH:C009560': 'trigonelline',
 'MESH:C512542': 'thymoglobulin',
 'CHEBI:CHEBI:9429': 'tectorigenin',
 'HGNC:7094': 'MICE',
 'CHEBI:CHEBI:75431': 'ghrelin',
 'FPLX:GAP': 'GAP',
 'CHEBI:CHEBI:28835': 'testosterone 17-glucosiduronic acid',
 'MESH:D013818': 'Thermogravimetry',
 'MESH:

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-01 01:54:33] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl
ERROR: [2020-11-01 01:59:02] adeft.gui.ground - Exception on /ground_add [POST]
Traceback (most recent call last):
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  F

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'tara gum': 'MESH:C038612',
  'target gene': 'ungrounded',
  'tectorigenin': 'CHEBI:CHEBI:9429',
  'test group': 'ungrounded',
  'testosterone glucuronide': 'CHEBI:CHEBI:28835',
  'tg': 'ungrounded',
  'tg mice': 'ungrounded',
  'tg101209': 'CHEBI:CHEBI:90304',
  'tgfbr2flox flox': 'ungrounded',
  'thapsigargin': 'CHEBI:CHEBI:9516',
  'therapy group': 'ungrounded',
  'thermogravimetric': 'ungrounded',
  'thermogravimetric analysis': 'ungrounded',
  'thermogravimetry': 'MESH:D013818',
  'thioglycerol': 'MESH:C009465',
  'thioglycollate': 'MESH:D013864',
  'thioglycollate broth': 'MESH:D013864',
  'thioguanine': 'CHEBI:CHEBI:9555',
  'thrombin generation': 'ungrounded',
  'thymine glycol': 'CHEBI:CHEBI:29128',
  'thymoglobulin': 'MESH:C512542',
  'thyroglobulin': 'HGNC:11764',
  'thyroid gland': 'MESH:D013961',
  'timothy grass': 'MESH:D031784',
  'total gastrectomy': 'MESH:D005743',
  'total ghrelin': 'CHEBI:CHEBI:75431',
  'total ginsenosides': 'CHEBI:CHEBI:74978',
  'total glutathio

In [11]:
grounding_map, names, pos_labels = [{'tara gum': 'MESH:C038612',
  'target gene': 'ungrounded',
  'tectorigenin': 'CHEBI:CHEBI:9429',
  'test group': 'ungrounded',
  'testosterone glucuronide': 'CHEBI:CHEBI:28835',
  'tg': 'ungrounded',
  'tg mice': 'ungrounded',
  'tg101209': 'CHEBI:CHEBI:90304',
  'tgfbr2flox flox': 'ungrounded',
  'thapsigargin': 'CHEBI:CHEBI:9516',
  'therapy group': 'ungrounded',
  'thermogravimetric': 'ungrounded',
  'thermogravimetric analysis': 'ungrounded',
  'thermogravimetry': 'MESH:D013818',
  'thioglycerol': 'MESH:C009465',
  'thioglycollate': 'MESH:D013864',
  'thioglycollate broth': 'MESH:D013864',
  'thioguanine': 'CHEBI:CHEBI:9555',
  'thrombin generation': 'ungrounded',
  'thymine glycol': 'CHEBI:CHEBI:29128',
  'thymoglobulin': 'MESH:C512542',
  'thyroglobulin': 'HGNC:11764',
  'thyroid gland': 'MESH:D013961',
  'timothy grass': 'MESH:D031784',
  'total gastrectomy': 'MESH:D005743',
  'total ghrelin': 'CHEBI:CHEBI:75431',
  'total ginsenosides': 'CHEBI:CHEBI:74978',
  'total glutathione': 'CHEBI:CHEBI:16856',
  'toxoplasma gondii': 'ungrounded',
  'training group': 'ungrounded',
  'transgenic': 'ungrounded',
  'transgenic pgc 1α overexpression': 'ungrounded',
  'transgenic rats': 'MESH:D055647',
  'transglutaminase': 'MESH:D011503',
  'transglycosylase': 'PF:PF00912',
  'transglycosylation': 'ungrounded',
  'transient grating': 'ungrounded',
  'transplant glomerulopathy': 'ungrounded',
  'transtricuspid gradient': 'ungrounded',
  'treatment gap': 'ungrounded',
  'treatment group': 'ungrounded',
  'triacyglycerol': 'CHEBI:CHEBI:17855',
  'triacylglyceride': 'CHEBI:CHEBI:17855',
  'triacylglycerol': 'CHEBI:CHEBI:17855',
  'trigeminal': 'MESH:D012668',
  'trigeminal ganglia': 'MESH:D012668',
  'trigeminal ganglion': 'MESH:D012668',
  'trigly ceride': 'CHEBI:CHEBI:17855',
  'triglyceride': 'CHEBI:CHEBI:17855',
  'triglyceride content': 'CHEBI:CHEBI:17855',
  'triglycide': 'CHEBI:CHEBI:17855',
  'triglycéride': 'CHEBI:CHEBI:17855',
  'triglyeride': 'CHEBI:CHEBI:17855',
  'trigonelline': 'MESH:C009560',
  'tripterygium glycosides': 'ungrounded',
  'troglitazone': 'CHEBI:CHEBI:9753',
  'trophoblast giant': 'BTO:BTO:0005362',
  'typhus group': 'ungrounded'},
 {'MESH:C038612': 'tara gum',
  'CHEBI:CHEBI:9429': 'tectorigenin',
  'CHEBI:CHEBI:28835': 'testosterone 17-glucosiduronic acid',
  'CHEBI:CHEBI:90304': 'TG101209',
  'CHEBI:CHEBI:9516': 'Thapsigargin',
  'MESH:D013818': 'Thermogravimetry',
  'MESH:C009465': 'thioglycerol',
  'MESH:D013864': 'Thioglycolates',
  'CHEBI:CHEBI:9555': 'tioguanine',
  'CHEBI:CHEBI:29128': 'thymine glycol',
  'MESH:C512542': 'thymoglobulin',
  'HGNC:11764': 'TG',
  'MESH:D013961': 'Thyroid Gland',
  'MESH:D031784': 'Phleum',
  'MESH:D005743': 'Gastrectomy',
  'CHEBI:CHEBI:75431': 'ghrelin',
  'CHEBI:CHEBI:74978': 'ginsenoside',
  'CHEBI:CHEBI:16856': 'glutathione',
  'MESH:D055647': 'Rats, Transgenic',
  'MESH:D011503': 'Transglutaminases',
  'PF:PF00912': 'Transglycosylase',
  'CHEBI:CHEBI:17855': 'triglyceride',
  'MESH:D012668': 'Trigeminal Ganglion',
  'MESH:C009560': 'trigonelline',
  'CHEBI:CHEBI:9753': 'troglitazone',
  'BTO:BTO:0005362': 'trophoblast giant cell'},
 ['CHEBI:CHEBI:17855', 'CHEBI:CHEBI:9516', 'HGNC:11764']]

In [12]:
excluded_longforms = ['tg']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [17]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [18]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [19]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-01 02:35:28] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-01 02:39:07] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9827130463339534 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [20]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:17855': 4976,
  'MESH:D012668': 264,
  'HGNC:11764': 99,
  'CHEBI:CHEBI:9555': 34,
  'CHEBI:CHEBI:9516': 719,
  'MESH:D013864': 35,
  'ungrounded': 513,
  'CHEBI:CHEBI:29128': 4,
  'MESH:D011503': 72,
  'MESH:C009465': 4,
  'MESH:D013961': 4,
  'MESH:D005743': 11,
  'MESH:D055647': 3,
  'CHEBI:CHEBI:9753': 9,
  'BTO:BTO:0005362': 13,
  'CHEBI:CHEBI:16856': 7,
  'MESH:C512542': 4,
  'CHEBI:CHEBI:74978': 9,
  'CHEBI:CHEBI:90304': 2,
  'PF:PF00912': 3,
  'MESH:D031784': 3,
  'CHEBI:CHEBI:28835': 3,
  'CHEBI:CHEBI:9429': 3,
  'MESH:C009560': 4,
  'CHEBI:CHEBI:75431': 3,
  'MESH:D013818': 3,
  'MESH:C038612': 3},
 'f1': {'mean': 0.982713, 'std': 0.001813},
 'precision': {'mean': 0.975161, 'std': 0.00264},
 'recall': {'mean': 0.990508, 'std': 0.001336},
 'MESH:D013818': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D012668': {'f1': {'mean': 0.971783, 'std': 0.020557},
  'pr': {'mean': 0.973

In [21]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [22]:
disamb.dump(model_name, results_path)

In [23]:
print(disamb.info())

Disambiguation model for TG

Produces the disambiguations:
	Gastrectomy	MESH:D005743
	Phleum	MESH:D031784
	Rats, Transgenic	MESH:D055647
	TG*	HGNC:11764
	TG101209	CHEBI:CHEBI:90304
	Thapsigargin*	CHEBI:CHEBI:9516
	Thermogravimetry	MESH:D013818
	Thioglycolates	MESH:D013864
	Thyroid Gland	MESH:D013961
	Transglutaminases	MESH:D011503
	Transglycosylase	PF:PF00912
	Trigeminal Ganglion	MESH:D012668
	ghrelin	CHEBI:CHEBI:75431
	ginsenoside	CHEBI:CHEBI:74978
	glutathione	CHEBI:CHEBI:16856
	tara gum	MESH:C038612
	tectorigenin	CHEBI:CHEBI:9429
	testosterone 17-glucosiduronic acid	CHEBI:CHEBI:28835
	thioglycerol	MESH:C009465
	thymine glycol	CHEBI:CHEBI:29128
	thymoglobulin	MESH:C512542
	tioguanine	CHEBI:CHEBI:9555
	triglyceride*	CHEBI:CHEBI:17855
	trigonelline	MESH:C009560
	troglitazone	CHEBI:CHEBI:9753
	trophoblast giant cell	BTO:BTO:0005362

Class level metrics:
--------------------
Grounding                          	Count	F1     
                       triglyceride*	4976	0.98733
              

In [24]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1